<a href="https://colab.research.google.com/github/JunyuYan/Pytorch-Learning-Materials/blob/main/Quick_Start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topic 1 Data Processing:**

Two primitives to work with data:
> torch.utils.data.DataLoader

> torch.utils.data.Dataset

In [25]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Pytorch provides domain-specific libraries as such as TorchText, TorchVision, and TorchAudio, all of which include datasets. In this tutorial, we take TorchVision as an example.

In [26]:
# Download data from open datasets
# For TorchVision, each dataset includes two augments: transform and target_transform
# to modify the samples and labels respectively
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

DataLoader wraps an iterable over our datasets, and supports automatic batching, sampling, shuffling and multiprocess data loading. The dataset
can be passed as an augment to DataLoader

In [27]:
batch_size = 64

# Create data loaders
# Each element in the dataloader iterable will return a batch of 64 features and labels
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}") # 一个batch数据大小
  print(f"Shape of y: {y.shape} {y.dtype}") # 一个batch标签大小
  break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


**Topic 2: Create Model**


Create a class that inherits from nn.Module


\_\_init\_\_: define the layers of the network


forward: specify how data will pass through the network


device: define how to accelerate operations

In [28]:
from torch.nn.modules.linear import Linear
# Create a simple neural network
# Define the accelerator for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define the model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**Topic 3: Train and Test the model**


Training: The model makes predictions on the training data, and then
backpropogate the prediction error to adjust the model's parameters.
The training process is conducted over several iterations (epochs). During
wach epoch, the model learns to make better predictions.  


Testing: Evaluate the model's performance on the new test dataset


In [29]:
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [30]:
# Define training process
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropogate
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Print loss for every 100 batchs
    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1)*len(X)
      print(f"loss:{loss:>7f} [current:{current:>5d}/{size:>5d}]")

In [31]:
# Define test process
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
       X, y = X.to(device), y.to(device)
       pred = model(X)
       test_loss += loss_fn(pred, y).item()
       correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [32]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("done!")

Epoch 1
---------------------------
loss:2.297778 [current:   64/60000]
loss:2.281790 [current: 6464/60000]
loss:2.271369 [current:12864/60000]
loss:2.267102 [current:19264/60000]
loss:2.245389 [current:25664/60000]
loss:2.226614 [current:32064/60000]
loss:2.230417 [current:38464/60000]
loss:2.200032 [current:44864/60000]
loss:2.189414 [current:51264/60000]
loss:2.165955 [current:57664/60000]
Test Error: 
 Accuracy: 51.1%, Avg loss: 2.158766 

Epoch 2
---------------------------
loss:2.166434 [current:   64/60000]
loss:2.150142 [current: 6464/60000]
loss:2.103571 [current:12864/60000]
loss:2.113746 [current:19264/60000]
loss:2.066442 [current:25664/60000]
loss:2.018605 [current:32064/60000]
loss:2.038460 [current:38464/60000]
loss:1.967390 [current:44864/60000]
loss:1.954375 [current:51264/60000]
loss:1.892532 [current:57664/60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.890121 

Epoch 3
---------------------------
loss:1.924523 [current:   64/60000]
loss:1.884631 [current: 6464/600

**Topic 4: Save and Load the model**


Save the model (save model's parameters):
torch.save(model.state_dict(), "model.pth")


Load the model (recreate the model and load model state dictionary):
torch.load("model.pth")


In [33]:
torch.save(model.state_dict(), "model.pth")
print("Successfully save the pytorch model")

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Successfully save the pytorch model


<All keys matched successfully>